In [1]:
import argparse
from prepare_train_val import get_split
from dataset import RoboticsDataset
import cv2
from models import UNet16, LinkNet34, UNet11, UNet, AlbuNet
import torch
from pathlib import Path
from tqdm import tqdm
import numpy as np
import utils
import prepare_data
from torch.utils.data import DataLoader
from torch.nn import functional as F
from prepare_data import (original_height,
                          original_width,
                          h_start, w_start
                          )
from albumentations import Compose, Normalize

In [45]:
from models import UNet11, LinkNet34, UNet, UNet16, AlbuNet
from model1 import LinkNet34_modified
from unetplusplus import UnetPlusPlus

In [52]:
model = UNet16(num_classes=1)
model_name = 'UNet16'
model_path = f'data/models/{model_name}/{model_name}.pt'
problem_type = 'binary'

state = torch.load(str(model_path))
state = {key.replace('module.', ''): value for key, value in state['model'].items()}
model.load_state_dict(state)

if torch.cuda.is_available():
    model.cuda()

/tmp/ipykernel_35329/4017577036.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.load(str(model_path))


In [53]:
import time
input_data = torch.randn(1, 3, 1024, 1280).cuda()

torch.cuda.synchronize()  # 同步GPU
start_time = time.time()
output = model(input_data)
torch.cuda.synchronize()  # 同步GPU
end_time = time.time()
print(end_time-start_time)

0.04085540771484375


In [55]:
from torch.profiler import profile, record_function, ProfilerActivity

with profile(activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA], record_shapes=True) as prof:
    with record_function("model_inference"):
        model(input_data)

print(prof.key_averages().table(sort_by="cuda_time_total"))

-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                        model_inference         0.00%       0.000us         0.00%       0.000us       0.000us      40.684ms       100.33%      40.684ms      40.684ms             1  
                                        model_inference         5.15%       2.342ms        31.39%      14.277ms      14.277ms       0.000us         0.00%      40.551ms      40.551ms             1  
         